In [ ]:
# Script to normalize and deblaze APF fits spectra. Modified from Jackie Telson's deblazing function.
# bstar_deblazed function -> returns deblazed, normalized spectrum. Uses blaze functions derived from spectra of B-stars 
#                     for each order.
# median_fit_deblazed function -> returns deblazed, normalized spectrum. Uses median fitting and masking of broad features to 
#                     derive blaze function for each order. -- NOT RECOMMENDED --


# output: normalized and deblazed spectrum
#NOTE: Must have 'apf_wav copy.fits' in directory

#NOTE: Must either: 1) have 'bstar_blaze_funcs.csv' which contains the APF blaze functions for each order,
#                      derived from B-star spectra, in directory 
#                or 2) have a directory of B-star spectra called 'bstars' containing the
#                      B-star spectra you would like to use to produce the blaze function, and first run
#                      median_fit_deblazed with is_bstar = True for each order (and save all orders to above csv)
#                      to create the file. May want to change the median filter window to 151 for this purpose.

import numpy as np
import astropy.io.fits as pf
import pandas as pd 
from matplotlib import pyplot as plt
from scipy import signal as sc


def median_fit_deblazed(file, order, is_bstar = False): # NOT RECOMMENDED, but may be useful for future customization
                                                        # of deblaze function for specific orders
    header = file[0].header
    image = file[0].data
    
    wave = pf.open('apf_wav copy.fits')
    wave_values = wave[0].data
    
    x = wave_values[order,0:4600] 
    y = image[order,0:4600]
    
    # normalize y values to 1
    y_norm = y/np.percentile(np.sort(y),99)
    
    # IF RUNNING SCRIPT TO PRODUCE BLAZE FUNCTION FROM BSTARS
    if is_bstar:
        y_medfit = sc.medfilt(y_norm, kernel_size = 151)
        all_bstar_blaze = y_medfit
        return all_bstar_blaze # return blaze func for this order, exit the function
        
    # 'mask' over known broad features
    y_medfit2 = sc.medfilt(y_norm, kernel_size = 601) 
    if order == 45:
        # Na-D features
        replace =np.arange(y_medfit2[1083],y_medfit2[1986],(y_medfit2[1986]-y_medfit2[1083])/(1986-1083))
        new_y_medfit2 = np.concatenate((y_medfit2[:1083],replace,y_medfit[1986:]))
        y_medfit2 = new_y_medfit2
    if order == 34:
        #first and second features
        replace =np.arange(y_medfit2[1233],y_medfit2[1849],(y_medfit2[1849]-y_medfit2[1233])/(1849-1233))
        new_y_medfit2 = np.concatenate((y_medfit2[:1233],replace,y_medfit[1849:]))
        y_medfit2 = new_y_medfit2
        #third feature
        replace =np.arange(y_medfit2[2063],y_medfit2[2508],(y_medfit2[2508]-y_medfit2[2063])/(2508-2063))
        new_y_medfit2 = np.concatenate((y_medfit2[:2063],replace,y_medfit2[2508:]))
        y_medfit2 = new_y_medfit2

    # deal with ends -- otherwise will get spike at ends as artifact of deblazing
    weights = np.arange(0,100)/100
    blaze_func_left = y_norm[0:100] - np.multiply((y_norm[0:100]-y_medfit2[0:100]), weights)
    blaze_func_right = y_norm[4600-100:4600] + np.multiply((y_norm[4600-100:4600]-y_medfit2[4600-100:4600]), weights-1)
    blaze = np.concatenate((blaze_func_left, y_medfit2[100:4600-100], blaze_func_right))
    
    # register
    dl_l = (wave_values[order,2844:2845]-wave_values[order,2843:2844])/wave_values[order,2843:2844]
    v = (dl_l*3*10**5)[0]
    lambda1 = wave_values[order,0]
    shifted = np.array([lambda1])
    for i in np.arange(0,4599):
        new_lambda = shifted[i] + (wave_values[order,i+1])*v/(3*10**5)
        shifted = np.append(shifted, new_lambda)
        
    # plot
    plot = False
    if is_bstar:
        plot = False
    save = False
    if plot == True:
        plt.figure()
        plt.plot(x,y_norm)
        plt.plot(x,blaze)
        plt.plot(x,y_norm/blaze, linewidth=0.5)
        plt.title('Blaze function (order ' + str(order) +')')
        plt.ylabel('Intensity')
        plt.xlabel('Wavelength [A]')
        plt.legend(['normalized',' median fit blaze func','deblazed'])
        if save == True:
            plt.savefig('Testing blaze function (order ' + str(order) +').png')
    
    print('Finished deblaze')
    
    return y_norm/blaze


def bstar_deblazed(file, order):
    
    # read in blaze functions
    all_bstar_blaze = pd.read_csv('bstar_blaze_funcs.csv').iloc[:,1:].values
    
    header = file[0].header
    image = file[0].data
    
    wave = pf.open('apf_wav copy.fits')
    wave_values = wave[0].data
    
    x = wave_values[order,0:4600] 
    y = image[order,0:4600]
    
    # normalize y values to 1
    y_norm = y/np.percentile(np.sort(y),99)
    
    # B-STAR DEBLAZE
    # and deal with ends -- otherwise will get spike at ends as artifcat of deblazing
    weights = np.arange(0,100)/100
    blaze_func_left_bstars = y_norm[0:100] - np.multiply((y_norm[0:100]-all_bstar_blaze[order-30][0:100]), weights) 
    blaze_func_right_bstars = y_norm[4600-100:4600] + np.multiply((y_norm[4600-100:4600]-all_bstar_blaze[order-30][4600-100:4600]), weights-1)
    bstar_blaze = np.concatenate((blaze_func_left_bstars, all_bstar_blaze[order-30][100:4600-100], blaze_func_right_bstars))
    bstar_deblazed = y_norm/bstar_blaze
    bstar_deblazed_norm = bstar_deblazed/np.percentile(np.sort(bstar_deblazed),99) # renormalize
    
    # register 
    # NOTE: This does not produce a registered wavelength scale, but instead a scale 
    #       in which a relative velocity of v m/s produces a contanst shift along the wavelength 
    #       scale for all wavelenghts.
    dl_l = (wave_values[order,2844:2845]-wave_values[order,2843:2844])/wave_values[order,2843:2844] 
    v = (dl_l*3*10**5)[0]
    lambda1 = wave_values[order,0]
    shifted = np.array([lambda1])
    for i in np.arange(0,4599):
        new_lambda = shifted[i] + (wave_values[order,i+1])*v/(3*10**5)
        shifted = np.append(shifted, new_lambda)
    
    # plot deblazing results
    plot = False
    save = False
    if plot == True:
        plt.figure()
        plt.plot(x,y_norm)
        plt.plot(x,bstar_blaze)
        plt.plot(x,bstar_deblazed_norm, linewidth=0.5)
        plt.title('Blaze function (order ' + str(order) +')')
        plt.ylabel('Intensity')
        plt.xlabel('Wavelength [A]')
        plt.legend(['normalized','bstar blaze func','deblazed'])
        if save == True:
            plt.savefig('Testing blaze function (order ' + str(order) +').png')
    
    print('Finished deblaze')
        
    return bstar_deblazed_norm
